In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import os

In [2]:
notebook_path = Path(os.getcwd())
root_path = notebook_path.parent.absolute()
os.chdir(root_path)
str(root_path)

'c:\\Users\\arsen\\Documents\\natixis_challenge'

### Loading datasets

In [4]:
df_cockpit = pd.read_csv('data/cockpit_20221221.csv').drop('Unnamed: 0', axis=1)
df_item_info = pd.read_csv('data/item_info_20221221.csv').drop('Unnamed: 0', axis=1)
df_item_trend = pd.read_csv('data/item_trend_20221221.csv').drop('Unnamed: 0', axis=1)
df_mycloud = pd.read_csv('data/mycloud_20221221.csv').drop('Unnamed: 0', axis=1)
df_tmp_hosts_zabbix = pd.read_csv('data/tmp_hosts_zabbix_20221221.csv').drop('Unnamed: 0', axis=1)

## Studying high capacity for consecutive days

In [8]:
df_item_trend.sample(1)

,itemid,clock,value_min,value_avg,value_max,item_type
2373718,100100054579792,2022-10-12,3.315988,5.96734,72.349406,cpu


In [35]:
usage_threshold = 90
value_type = "value_max"

In [47]:
def get_frequency_above_threshold(df_item_trend, value_type, usage_threshold):
    df = df_item_trend.copy()  

    # 0 or 1 if "value is above threshold"
    df["above_threshold (%)"] = (df[value_type] > usage_threshold)*1

    # Get frequency of "value is above threshold"
    df = df[["itemid", "above_threshold (%)"]]\
        .groupby("itemid")\
        .mean()*100

    return df.reset_index()

get_frequency_above_threshold(df_item_trend, value_type, usage_threshold).sample()

,itemid,above_threshold (%)
6094,100100052798375,6.161137


In [118]:
def get_count_of_servers_above_thresholds_during(df_item_trend, value_type, threshold, during=2):
    df = df_item_trend.copy()

    df["above_threshold"] = df[value_type] >= threshold
    
    df["previous_n_days_underthreshold"] = pd.Series(
        np.logical_and.reduce([df["above_threshold"].shift(i) for i in range(during)])
    )
    
    return df

get_count_of_servers_above_thresholds_during(df_item_trend, value_type="value_max", threshold=64, during=3).sample()

,itemid,clock,value_min,value_avg,value_max,item_type,above_threshold,previous_n_days_underthreshold
1910223,100100040896446,2022-10-15,66.39445,70.606057,81.067274,mem,True,True


In [122]:
print(get_count_of_servers_above_thresholds_during(df_item_trend, value_type="value_max", threshold=100, during=2)\
    ["previous_n_days_underthreshold"].mean())

print("\n", 
"There are 1.14 % of servers whose max_value has gone above threshold for 2 consecutive days"
)

0.011408439301603937

 There are 1.14 % of servers whose max_value has gone above threshold for 2 consecutive days


In [135]:
print(get_count_of_servers_above_thresholds_during(df_item_trend, value_type="value_min", threshold=90, during=3)\
    ["previous_n_days_underthreshold"].mean()*100)

print("There are 2.16 % of servers with min_value over 90 for 3 consecutive days")

2.161181095620915
There are 2.16 % of servers with min_value over 90 for 3 consecutive days


In [136]:
print("An example of server with very high value_min for consecutive days ")
df_item_trend.iloc[401490:401497]

An example of server with very high value_min for consecutive days


,itemid,clock,value_min,value_avg,value_max,item_type
401490,100100013303178,2021-10-27,91.072593,92.218769,92.859655,mem
401491,100100013303178,2021-10-28,91.782055,92.321963,93.307115,mem
401492,100100013303178,2021-10-29,91.955723,92.236123,93.211483,mem
401493,100100013303178,2021-10-30,92.061129,93.513694,94.652618,mem
401494,100100013303178,2021-10-31,91.501026,92.707906,97.263741,mem
401495,100100013303178,2021-11-01,91.895235,92.349869,93.723365,mem
401496,100100013303178,2021-11-02,91.857731,92.105027,93.106482,mem


In [139]:
print(get_count_of_servers_above_thresholds_during(df_item_trend, value_type="value_min", threshold=100, during=3)\
    ["previous_n_days_underthreshold"].mean()*100)

print("There are 0.967 % of servers with min_value over 100 for 3 consecutive days")

0.009673648851202468
There are 0.967 % of servers with min_value over 100 for 3 consecutive days


In [142]:
print("An example of server cmpletely saturated for consecutive days ")
df_item_trend.iloc[250067-3:250067+3]

An example of server cmpletely saturated for consecutive days 


,itemid,clock,value_min,value_avg,value_max,item_type
250064,100100014005240,2022-11-28,100.0,100.0,100.0,cpu
250065,100100014005240,2022-11-29,100.0,100.0,100.0,cpu
250066,100100014005240,2022-11-30,100.0,100.0,100.0,cpu
250067,100100014005240,2022-12-01,100.0,100.0,100.0,cpu
250068,100100014005240,2022-12-02,100.0,100.0,100.0,cpu
250069,100100014005240,2022-12-03,100.0,100.0,100.0,cpu
